## 偏函数Partial Function
### 从使用case语句构造匿名函数谈起
在Scala里，我们可以使用case语句来创建一个匿名函数（函数字面量），这有别于一般的匿名函数创建方法。来看个例子：

In [1]:
List(1,2,3) map {case i:Int => i+1}

res0: List[Int] = List(2, 3, 4)

这很有趣，`case i:Int=>i+1`构建的匿名函数等同于`(i:Int)=>i+1`，也就是下面这个样子：

In [2]:
List(1,2,3) map {(i:Int) => i+1}

res1: List[Int] = List(2, 3, 4)

《Scala In Programming》一书对独立的case语句作为匿名函数（函数字面量）有权威的解释：

Essentially, a case sequence is a function literal, only more general. Instead of having a single entry point and list of parameters, a case sequence has multiple entry points, each with their own list of parameters. Each case is an entry point to the function, and the parameters are specified with the pattern. 

一个case语句就是一个独立的匿名函数，如果有一组case语句的话，从效果上看，构建出的这个匿名函数会有多种不同的参数列表，每一个case对应一种参数列表，参数是case后面的变量声明，其值是通过模式匹配赋予的。

### 使用case语句构造匿名函数的“额外”好处
使用case语句构造匿名函数是有“额外”好处的，这个“好处”在下面这个例子中得到了充分的体现：

In [3]:
//List(1, 3, 5, "seven") map { case i: Int => i + 1 } // won't work
// scala.MatchError: seven (of class java.lang.String)

List(1, 3, 5, "seven") collect { case i: Int => i + 1 }

res2: List[Int] = List(2, 4, 6)

在这个例子中：传递给map的case语句构建的是一个普通的匿名函数，在把这个函数适用于”seven”元素时发生了类型匹配错误。而对于collect,它声明接受的是一个偏函数：PartialFunction，传递的case语句能良好的工作说明这个case语句被编译器自动编译成了一个PartialFunction！这就是case语句“额外”的好处：**case语句（组合）除了可以被编译为匿名函数（类型是FunctionX，在Scala里，所有的函数字面量都是一个对象，这个对象的类型是FunctionX），还可以非常方便的编译为一个偏函数PartialFunction！（注意：PartialFunction同时是Function1的子类）编译器会根据调用处的函数类型声明自动帮我们判定如何编译这个case语句（组合）。**

上面我们直接抛出了偏函数的概念，这会让人头晕，我们可以只从collect这个示例的效果上去理解偏函数：它只对会作用于指定类型的参数或指定范围值的参数实施计算，超出它的界定范围之外的参数类型和值它会忽略（未必会忽略，这取决于你打算怎样处理）。就像上面例子中一样，`case i: Int => i + 1`只声明了对Int参数的处理，在遇到”seven”元素时，不在偏函数的适用范围内，所以这个元素被忽略了。

### 正式认识偏函数Partial Function
如同在一开始的例子中那样，我们手动实现了一个与`case i:Int=>i+1`等价的那个匿名函数`(i:Int)=>i+1`,那么在上面的collect方法中使用到的`case i: Int => i + 1`它的等价函数是什么呢？显然，不可能是`(i:Int)=>i+1`了，因为我们已经解释了，collect接受的参数类型是`PartialFunction[Any,Int]`,而不是`(Int)=>Int`。 那个case语句对应的偏函数具体是什么样的呢？来看：

In [4]:
val inc = new PartialFunction[Any, Int] {
    def apply(any: Any) = any.asInstanceOf[Int]+1
    def isDefinedAt(any: Any) = any.isInstanceOf[Int]
}

inc: AnyRef with PartialFunction[Any, Int] = <function1>

In [5]:
List(1, 3, 5, "seven") collect inc

res4: List[Int] = List(2, 4, 6)

PartialFunction特质规定了两个要实现的方法：apply和isDefinedAt，isDefinedAt用来告知调用方这个偏函数接受参数的范围，可以是类型也可以是值，在我们这个例子中我们要求这个inc函数只处理Int型的数据。apply方法用来描述对已接受的值如何处理，在我们这个例子中，我们只是简单的把值+1，注意，非Int型的值已被isDefinedAt方法过滤掉了，所以不用担心类型转换的问题。

上面这个例子写起来真得非常笨拙，和前面的case语句方式比起来真是差太多了。这个例子从反面展示了：通过case语句组合去是实现一个偏函数是多么简洁。实际上case语句组合与偏函数的用意是高度贴合的，所以使用case语句组合是最简单明智的选择，同样是上面的inc函数，换成case去写如下：

In [6]:
def inc: PartialFunction[Any, Int] = { case i: Int => i + 1 }

List(1, 3, 5, "seven") collect inc

defined function inc
res5_1: List[Int] = List(2, 4, 6)

In [7]:
//还是这样写常见
List(1, 3, 5, "seven") collect { case i: Int => i + 1 }

res6: List[Int] = List(2, 4, 6)

### Case语句是如何被编译成偏函数的
关于这个问题在《Programming In Scala》中有较为详细的解释。对于这样一个使用case写在的偏函数：
```scala
val second: PartialFunction[List[Int],Int] = {
    case x :: y :: _ => y
}
```

In fact, such an expression gets ranslated by the Scala compiler to a partial function by translating the patterns twice—once for the implementation of the real function, and once to test whether the function is defined or not. For instance, the function literal `{ case x :: y :: _ => y }`above gets translated to the following partialfunction value:
```scala
new PartialFunction[List[Int], Int] {
    def apply(xs: List[Int]) = xs match {
        case x :: y :: _ => y
    }
    def isDefinedAt(xs: List[Int]) = xs match {
        case x :: y :: _ => true
        case _ => false
    }
}
```

### 为什么偏函数需要抽象成一个专门的Trait
首先，在Scala里，一切皆对象，函数字面量（匿名函数）也不例外！这也是为什么我们可以把函数字面量赋给一个变量的原因, 是对象就有对应的类型，那么一个函数字面量的真实类型是什么呢？看下面这个例子：
```scala
scala> var inc = (x: Int) => x + 1
inc: Int => Int = <function1>

scala> inc.isInstanceOf[Function1[Int,Int]]
res0: Boolean = true
```

在Scala的scala包里，有一系列Function trait，它们实际上就是函数字面量作为“对象”存在时对应的类型。Function类型有多个版本，Function0表示无参数函数，Function1表示只有一个参数的函数，以此类推。至此我们解释的是一个普遍性问题：是函数就是对象，是对象就有类型。那么，接下来我们看一下偏函数又应该是什么样的一种“类型”？

从语义上讲，偏函数区别于普通函数的唯一特征就是：偏函数会自主地告诉调用方它的处理参数的范围，范围既可是值也可以是类型。针对这样的场景，我们需要给函数安插一种明确的“标识”，告诉编译器：这个函数具有这种特征。所以特质PartialFunction就被创建出来用于“标记”这类函数的，这个特质最主要的方法就是isDefinedAt！同时你也记得PartialFunction还是Function1的子类，所以它也要有apply方法，这是非常自然的，偏函数本身首先是一个函数嘛。

从另一个角度思考，偏函数的逻辑是可以通过普通函数去实现的，只是偏函数是更为优雅的一种方式，同时**偏函数特质PartialFunction的存在对调用方和实现方都是一种语义更加丰富的约定，比如collect方法声明使用一个偏函数就暗含着它不太可能对每一个元素进行操作，它的返回结果仅仅是针对偏函数“感兴趣”的元素计算出来的。**

### 为什么偏函数只能有一个参数？
为什么只有针对单一参数的偏函数，而不是像Function特质那样，拥有多个版本的PartialFunction呢？在刚刚接触偏函数时，这也让我感到费解，但看透了偏函数的实质之后就会觉得很合理了。我们说所谓的偏函数本质上是由多个case语句组成的针对每一种可能的参数分别进行处理的一种“结构较为特殊”的函数，那特殊在什么地方呢？对，就是case语句，前面我们提到，case语句声明的变量就是偏函数的参数，既然case语句只能声明一个变量，那么偏函数受限于此，也只能有一个参数！说到底，类型PartialFunction无非是为由一组case语句描述的函数字面量提供一个类型描述而已，case语句只接受一个参数，则偏函数的类型声明自然就只有一个参数。

但是，上这并不会对编程造成什么阻碍，如果你想给一个偏函数传递多个参数，完全可以把这些参数封装成一个Tuple传递过去！

## 偏函数
摘抄自《scala程序设计（第2版）》————2.4偏函数

我 们 来 讨 论 偏 函 数( PartialFunction) 的 性 质。 偏 函 数 之 所 以“偏”,原因在于它们并不处理所有可能的输入,而只处理那些能与至少一个 case 语句匹配的输入。

在偏函数中只能使用 case 语句,而整个函数必须用花括号包围。这与普通的函数字面量不同,普通函数字面量可以用花括号,也可以用圆括号包围。

如果偏函数被调用,而函数的输入却与所有语句都不匹配,系统就会抛出一个 MatchError 运行时错误。

我们可以用 isDefineAt 方法测试特定输入是否与偏函数匹配,这样偏函数就可以避免抛出MatchError 错误了。

偏函数可以如此“链式”连接: pf1 orElse pf2 orElse pf3... 。如果 pf1 不匹配,就会尝试 pf2 ,接着是 pf3 ,以此类推。如果以上偏函数都不匹配,才会抛出 MatchError 。

以下实例可以展示上述规则:
```scala
// src/main/scala/progscala2/typelessdomore/partial-functions.sc

val pf1: PartialFunction[Any,String] = { case s:String => "YES" }    // ➊
val pf2: PartialFunction[Any,String] = { case d:Double => "YES" }    // ➋

val pf = pf1 orElse pf2                                              // ➌

def tryPF(x: Any, f: PartialFunction[Any,String]): String =          // ➍ 
  try { f(x).toString } catch { case _: MatchError => "ERROR!" }

def d(x: Any, f: PartialFunction[Any,String]) =                      // ➎
  f.isDefinedAt(x).toString

println("      |   pf1 - String  |   pf2 - Double  |    pf - All")   // ➏
println("x     | def?  |  pf1(x) | def?  |  pf2(x) | def?  |  pf(x)")
println("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
List("str", 3.14, 10) foreach { x =>
  printf("%-5s | %-5s | %-6s  | %-5s | %-6s  | %-5s | %-6s\n", x.toString, 
    d(x,pf1), tryPF(x,pf1), d(x,pf2), tryPF(x,pf2), d(x,pf), tryPF(x,pf))
}
```
➊ 只匹配字符串的偏函数。  
➋ 只匹配 Double 数字的偏函数。  
➌ 将这两个函数结合,得到一个新的偏函数:既能匹配字符串,又能匹配 Double 数字。  
➍ 辅助函数:用于 try 一个偏函数,然后将可能产生的 MatchError 异常捕捉到。无论是否捕获异常,函数均返回一个字符串。  
➎ 辅助函数:使用了 isDefineAt ,返回值为字符串。  
➏ 使用了多个偏函数的链式组合,并将结果以表格的形式打印出来。  

其他代码对这 3 个偏函数输入不同的值,先调用 isDefineAt (结果显示在输出表中的 def?这一列),然后再尝试调用偏函数本身。输出为:
```
      |   pf1 - String  |   pf2 - Double  |    pf - All
x     | def?  |  pf1(x) | def?  |  pf2(x) | def?  |  pf(x)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
str   | true  | YES     | false | ERROR!  | true  | YES   
3.14  | false | ERROR!  | true  | YES     | true  | YES   
10    | false | ERROR!  | false | ERROR!  | false | ERROR!
```

未输入字符串时, pf1 将会失败;未输入 Double 数字时, pf2 会失败;如果给出整数,这两个函数均失败。组合后的函数 pf 对于字符串或者 Double 数字的输入均成功,但输入整数时仍将失败。

## 应该广泛使用case语句简化函数字面量的实现
这篇文章很短，但足以单独作为一篇文章存在，因为这里介绍的这个Scala编程技巧非常的实用和高效。

在Scala里我们需要大量地编写一些函数字面量作为参数传递给高阶函数，而很多时候，被要求提供的函数字面量的参数是一些复合类型，比如tuple，在函数字面量的实现体中，往往第一步就是通过模式匹配从复合类型中提取出嵌套的变量进行计算，例如下面计算元组两元素平方差的例子：

In [8]:
val list = Seq((1,2),(3,4))

list: Seq[(Int, Int)] = List((1, 2), (3, 4))

In [9]:
list.map{ tuple =>
    val (num1, num2) = tuple
    (num2 + num1) * (num2 - num1)
}

res8: Seq[Int] = List(3, 7)

在这个例子中我们使用的是常规的函数字面量的实现方式，而实际上我们应该广泛地使用case语句去定义函数字面量，因为它确实有很多“额外”的好处，因为**使用case定义函数字面量可以将参数列表声明与嵌套变量的提取合二为一，简化函数的实现**！，上面的例子如果使用case语句实现将会是这样的：

In [10]:
list.map{
    case (num1, num2) =>
    (num2 + num1) * (num2 - num1)
}

res9: Seq[Int] = List(3, 7)

不但需要单个参数的单个参数可以用偏函数，多个参数也可以用偏函数，把所有的参数当成一个元组就行，这可能是编译器提我们做好的，反正是可以用的，很方便。

In [11]:
//测试多个参数的情况
def test(x:Int, y:Int)( f:(Int, Int) => Int ):Int =  f(x,y)
test(2, 3){case (x,y) => x + y}

defined function test
res10_1: Int = 5

In [12]:
//参数更复杂的情况
def test2(x:(Int, Int), y:(Int, Int))( f:((Int, Int), (Int, Int)) => (Int, Int) ):(Int, Int) =  f(x,y)
test2((1, 2), (3, 4)){case ((x1,y1), (x2,y2)) => (x1+x2,y1+y2)}

defined function test2
res11_1: (Int, Int) = (4, 6)

In [13]:
//函数参数不是一个独立的参数列表的情况
def test3(x:Int, y:Int, f:(Int, Int) => Int ):Int =  f(x,y)
test3(2, 3, {case (x,y) => x + y})

defined function test3
res12_1: Int = 5

In [14]:
//函数参数不是一个独立的参数列表的情况
def test4(x:(Int, Int), y:(Int, Int), f:((Int, Int), (Int, Int)) => (Int, Int) ):(Int, Int) =  f(x,y)
test4((1, 2), (3, 4), {case ((x1,y1), (x2,y2)) => (x1+x2,y1+y2)})

defined function test4
res13_1: (Int, Int) = (4, 6)